Data Wrangling Yelp Business Data -- Reilly Proudsworth -- Capstone One

In order to view/clean the Yelp data, we need to start by reading it in from it's JSON file.  As always we import the needed packages, in this case we'll bring in pandas and numpy so we can work with our data in dataframe/array formats.

In [19]:
import pandas as pd
import numpy as np

Next we read in the JSON file.  The dataset isn't perfectly formatted and has a line of white space at the end of the document.  This requires we read in JSON line by line, which we accomplish by setting the 'lines' parameter to True.  The results are seen by looking at the head of our new dataframe: business_df. 

In [20]:
business_df = pd.read_json('yelp_academic_dataset_business.json', lines = True)
business_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,type
0,"227 E Baseline Rd, Ste J2","[BikeParking: True, BusinessAcceptsBitcoin: Fa...",0DI8Dt2PJp07XkVvIElIcQ,"[Tobacco Shops, Nightlife, Vape Shops, Shopping]",Tempe,"[Monday 11:0-21:0, Tuesday 11:0-21:0, Wednesda...",0,33.378214,-111.936102,Innovative Vapors,,85283,17,4.5,AZ,business
1,495 S Grand Central Pkwy,"[BusinessAcceptsBitcoin: False, BusinessAccept...",LTlCaCGZE14GuaUXUGbamg,"[Caterers, Grocery, Food, Event Planning & Ser...",Las Vegas,"[Monday 0:0-0:0, Tuesday 0:0-0:0, Wednesday 0:...",1,36.192284,-115.159272,Cut and Taste,,89106,9,5.0,NV,business
2,979 Bloor Street W,"[Alcohol: none, Ambience: {'romantic': False, ...",EDqCEAGXVGCH4FJXgqtjqg,"[Restaurants, Pizza, Chicken Wings, Italian]",Toronto,"[Monday 11:0-2:0, Tuesday 11:0-2:0, Wednesday ...",1,43.661054,-79.429089,Pizza Pizza,Dufferin Grove,M6H 1L5,7,2.5,ON,business
3,7014 Steubenville Pike,"[AcceptsInsurance: False, BusinessAcceptsCredi...",cnGIivYRLxpF7tBVR_JwWA,"[Hair Removal, Beauty & Spas, Blow Dry/Out Ser...",Oakdale,"[Tuesday 10:0-21:0, Wednesday 10:0-21:0, Thurs...",1,40.444544,-80.174540,Plush Salon and Spa,,15071,4,4.0,PA,business
4,321 Jarvis Street,"[BusinessAcceptsCreditCards: True, Restaurants...",cdk-qqJ71q6P7TJTww_DSA,"[Hotels & Travel, Event Planning & Services, H...",Toronto,None,1,43.659829,-79.375401,Comfort Inn,Downtown Core,M5B 2C2,8,3.0,ON,business


We want to create some dummy variables out of the categories column which can eventually be used to tune our regression model.  We need to check if there is missing data in the column before we attempt to manipulate it.  Looking at the length of business_df reveals we're working with more than 144,000 rows, far too many to just go through manually and visually inspect for missing values.  Running it through the pandas function notnull returns a boolean column showing which rows have data.  A value count on this result gives us a count of 143,747 rows with data and 325 without.  Typically we don't want to exclude data, however in this instance the count of empty rows is so small (about 0.25% of the total data) we can remove those rows without worry.

Subsetting business_df by the results of our notnull function returns a dataframe without the missing data.  We need to make a copy of that dataframe in order to not just work with a view of business_df, this new dataframe is named category_df.  Looking at the index of category_df we see that the index isn't sequential from 0 to 143746 as expected.  Using reset_index fixes this issue.

In [34]:
print(len(business_df))
print(pd.notnull(business_df.categories).value_counts())
category_df = business_df[pd.notnull(business_df.categories)].copy()
print(category_df.index)
category_df = category_df.reset_index()
print(category_df.index)
print(len(category_df))

144072
True     143747
False       325
Name: categories, dtype: int64
Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            144062, 144063, 144064, 144065, 144066, 144067, 144068, 144069,
            144070, 144071],
           dtype='int64', length=143747)
RangeIndex(start=0, stop=143747, step=1)
143747


In [22]:
category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143747 entries, 0 to 143746
Data columns (total 17 columns):
index           143747 non-null int64
address         143747 non-null object
attributes      127162 non-null object
business_id     143747 non-null object
categories      143747 non-null object
city            143747 non-null object
hours           102453 non-null object
is_open         143747 non-null int64
latitude        143747 non-null float64
longitude       143747 non-null float64
name            143747 non-null object
neighborhood    143747 non-null object
postal_code     143747 non-null object
review_count    143747 non-null int64
stars           143747 non-null float64
state           143747 non-null object
type            143747 non-null object
dtypes: float64(3), int64(3), object(11)
memory usage: 18.6+ MB


The next step is to extract the lists of categories for each business and create dummy variables.  In order to accomplish this we have to get a list of unique category names.  By interating through the categories column of category_df, we can grab one list element then we iterate through that list and append each category to a list.  Converting the resulting list to a set removes duplicate entries.  We store this list in the variable cats.

In [23]:
cats = []
for category_list in category_df.categories:
    for single_category in category_list:
        cats.append(single_category)

cats = list(set(cats))

Our dummy variables will be binary 0 or 1 so we can set them all to 0 to begin with.  To do so we create a numpy array filled with zeroes with a number of rows the length of category_df and columns the length of cats. Next we convert the array into a pandas dataframe with columns named the categories in cats and we set the index to match category_df.  Finally we join category_df and cols to create one dataframe.  The lengths of all lists, arrays, and dataframes are below for reference.

In [24]:
zeros = np.zeros(shape=(len(category_df),len(cats)))
cols = pd.DataFrame(zeros, columns=cats, index=category_df.index)
category_df = category_df.join(cols)

In [25]:
print(len(cats))
print(len(zeros))
print(len(cols))
print(len(category_df))
print(len(category_df.Restaurants))

1191
143747
143747
143747
143747


To get ones in the dataframe under the related column, we need to compare the categories for a business against the names of the columns and place a one where they match.  By iterating through the categories column and enumerating the result we can get a list of the business's categories and a row number.  We can then iterate through a list of the column names and compare if the column name is in the list of business categories and place a one when true.

Looking at the results we see that our final dataframe is the correct length, full of zeroes and ones for the appropriate businesses. Our last step is to write the dataframe to a csv for further work in conjunction with the other Yelp data.

In [ ]:
colname = category_df.columns.tolist()

In [27]:
for rownum, business_categories in enumerate(category_df.categories):
    for category_name in colname:
        if category_name in business_categories:
            category_df.loc[rownum, category_name] = 1

In [28]:
category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143747 entries, 0 to 143746
Columns: 1208 entries, index to Signmaking
dtypes: float64(1194), int64(3), object(11)
memory usage: 1.3+ GB


In [32]:
print(category_df.Restaurants.head())
print(category_df.Restaurants.tail())

0    0.0
1    0.0
2    1.0
3    0.0
4    0.0
Name: Restaurants, dtype: float64
143742    0.0
143743    0.0
143744    0.0
143745    1.0
143746    0.0
Name: Restaurants, dtype: float64


In [30]:
category_df.to_csv('business_data_with_dummy_variables.csv')